In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
!pwd
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')
# %load_ext autoreload
# %autoreload 2


from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv

C:\Users\Egor\Dropbox\GitHub\Mosaic\msq-domain\notebooks


In [2]:
import pickle

try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
    filename = '../resources/fx/EUR_JPY_GBP_till_SN.csv'
    quote_dict = qc_csv_helper.get_fx_quotes(filename)
    with open('data.pickle', 'wb') as f:\
        pickle.dump(quote_dict, f)

In [4]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_fx_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_fx_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)

my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp,
                         sym = my_quote.sym)
print(my_trade.sym)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  static = Instrument static singleton facade, sym = EUR=, ccy = ('EUR', 'USD'), isDeliverable = True, tenor = SPOT, venue = None, settle_date = 2017-06-26, notionals = (100, -111.97999999999999), holidayCities = None, date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x000001D2FEC40DA0>
EUR=


In [5]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [6]:
logging.getLogger().setLevel('INFO')
logging.info('staring command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes, trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([0, 10, 100]))\
            > print
        
run(stream)



INFO:root:staring command line test...
INFO:aiostreams.runner:Creating a runnable graph with source <aiostreams.operators.flatten.Flatten object at 0x000001D2FFDA6898> and sink <built-in function print>
INFO:aiostreams.runner:Attempting to run the graphs ["{'source': <aiostreams.operators.flatten.Flatten object at 0x000001D2FFDA6898>, 'sink': <built-in function print>, 'id': 'pipe_52714', 'logger': <logging.Logger object at 0x000001D2FFDA6BE0>, 'ignore_heartbeats': True, 'message_count': 0, 'persister_factory': <class 'aiostreams.persist.KafkaPersister'>, 'persistence_policy': None, 'log_level': 10, 'persister': None}; "]
INFO:aiostreams.runner:Graph pipe_52714 creating a coroutine for the event loop
INFO:aiostreams.runner:Finished running graphs ["{'source': <aiostreams.operators.flatten.Flatten object at 0x000001D2FFDA6898>, 'sink': <built-in function print>, 'id': 'pipe_52714', 'logger': <logging.Logger object at 0x000001D2FFDA6BE0>, 'ignore_heartbeats': True, 'message_count': 0, 'p